# <span style='color:red'>Quantitative Investing with Python</span>

### Professor Juhani Linnainmaa

Dartmouth College and Kepos Capital 

--- 

# **Topic 2:** Working with Pandas Dataframes

- Pandas is Python package
  - A package is a collection of additional objects that extends the capability of the basic Python
  - E.g., if you want to create nicer plots, you might use *matplotlib* package; if you want to do machine learning, you'd maybe use *sklearn*<br><br>
  
- Pandas is the most popular package for statistics
  - It has finance origins: 
  <br><br>  
  ```
  Wes McKinney started building what would become Pandas at AQR Capital while he was a researcher there from 2007 to 2010.
  ```
  <br>
- The data in Pandas are organized in ```DataFrame```s (or ```Series```)
- In this lecture we load some data and go through some basics of how to view, manipulate, and plot the data we have
- I cover **a lot** of ground
  - However, I'm confident in that *everything* that I do here is something that quantitative researchers/scientists working with Pandas would use daily
  - It will be useful to see just how flexible Pandas can be
  - In terms of quantitative investing, Python allows you to think about what you want to do instead of worrying about *how* you can implement your ideas<br><br>  
- A few words of comfort:
  1. You don't need to memorize things. When you first start working with Python, you'll constantly check the syntax and go back and see how something is done. But as you keep doing it, the things that matter will stick.
  2. There is no real-time coding test in real life so you can always look things up
     - A Google search is typically your friend. Nowadays an LLM, possibly trained specifically for coding, is a very powerful tool.

We begin by importing a few packages that we need

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

Note: I import the ```datetime``` and ```timedelta``` functions from ```datetime```. I could just as well import the entire package by writing ```import datetime as dt``` and then refer to these functions ```dt.datetime``` and ```dt.timedelta```. Both approaches are just fine.

## Use Yahoo Finance API to download daily stock prices

- We need to define 
  1. "ticker symbols" for the stocks we want
  2. the sample period we want to cover
- Tickers will be a list of strings
  - Whatever you trade, be it stocks, currencies, commodities,... you need to know the asset codes
- The start and end dates are "datetimes", which I construct using the datetime package
- NOTE 1: Since 2024, Yahoo Finance appears to automatically adjust prices for at least splits
- NOTE 2: On the last row in the code below, I just write ```stock_data.head()```, which keeps the first five rows of the dataframe. I do **not** ask Python to print this. HOWEVER, in notebooks, the last thing that gets returned in the block code is always printed so the outcome is the same, except for formatting, as it would be if I were to write ```print(stock_data.head())``` 

In [ ]:
# Define the ticker symbols for Apple, Google, and Tesla
ticker_symbols = ['AAPL', 'GOOGL', 'TSLA']

# Define the start and end dates for the data
end_date = datetime(2024,12,31)
start_date = end_date - timedelta(days=5 * 365)  # Five years ago

# Download the data for each stock
stock_data = yf.download(ticker_symbols, start=start_date, end=end_date)

# Display the first few rows of the downloaded data
stock_data.head()

## Preliminary plotting of some data

- To get started, let me get the closing prices for TSLA and plot them
- Don't worry about *how* I do this in the code below just yet

In [ ]:
tsla = stock_data.loc[:,('Close', 'TSLA')]
tsla.plot(figsize=(16,8), title='Closing Price for Tesla')

# ```DataFrame```s

The main object for storing data in Pandas is a ```DataFrame```. You can think of it as being a 2D object with a horizontal and vertical axises. 
- These axises are indexed 0 and 1. The vertical axis is called "index" and the horizontal axis (across columns) is called "columns"

---

```DataFrame``` is an object just like anything else with its own methods
- I could write things such as ```stock_data.sum()``` or ```stock_data.shape``` or ```stock_data.join(...)``` to do stuff

---

We can get the contents of these indices from the stock_data object:  

In [ ]:
stock_data.index

In [ ]:
stock_data.columns

Our index has only one "level"; it is one series of dates.

Our columns, however, have two levels. This is known as a **MultiIndex**.
- When you have multiple levels, each column is identified as a ```tuple``` where the elements give the labels for each level
- Indices and columns can also have names associated with each level. For example, you can see above that in this ```MultiIndex```, the level corresponding to Close, High,... is called ```Price``` and the level corresponding to the tickers is called ```Ticker``` 

To start things simple, let's create a simpler dataframe that only keeps the "Close" information for each stock.

Method .xs() ("cross-section") let's us extract this info easily. I just need to tell (a) what label I want, (b) which axis I'm looking at, and (c) what level I'm looking at
- To be clear, there are many other ways of doing the same thing. I'll talk about some of them below.

In [ ]:
close = stock_data.xs('Close', axis=1, level=0)
close

If we want to do fancy visualizations, we might use some other Python packages

However, for most of the data work where the point is not about making the prettiest possible graph, Pandas integration with matplotlib is enough--and it is convenient

I can plot some/all of the data by just a calling the plot() method of the DataFrame:

In [ ]:
close.plot(figsize=(12,6), title='My title goes here!')

## Creating new columns into a DataFrame

You can create new columns based on the other columns

For example, your new column could be, say, the sum of some other columns  

In terms of syntax, we can think of columns as being a collection such as a dictionary or a list and we create a new entry for what we want:

In [ ]:
close['apple_plus_tesla'] = close['AAPL'] + close['TSLA']
close.tail(5)

There are also some simple methods for creating additional statistics. 

Let me first drop the non-sense column that is just created -- there is an intuitive drop-method for doing so -- and then create a price-weighted index of the three stocks

Note: When I drop the unnecessary column, I modify the dataframe *inplace*. I personally prefer to do it as

```df = df.drop(columns='col1')```

but it is just a personal preference.

In [ ]:
close.drop(columns='apple_plus_tesla', inplace=True)
close

Let me now ceate a new column that is just the average of the three (adjusted) stock prices each day:

In [ ]:
close['index'] = close.mean(axis=1)
close

Because I want to compute the average for each day, I'm taking the average over columns -- which is why there is an argument ```axis=1``` (columns)

What would happen if I put ```axis=0``` instead? Let's try it out!

In [ ]:
close.mean(axis=0)

So this is taking the average along the ```index``` (where we have the datas) -- so we have the average adjusted price over the full sample. If you had, e.g., stock returns in your data, you'd take average along this dimension to compute average returns

### Some plotting

What if I want to plot the data in just one column in the dataframe?

If I want to plot just the new ```index``` column, I can just refer to that column and plot it:

In [ ]:
print(close['index'].tail())
close['index'].plot()

Note that when I get just one column from a dataframe -- so that there is just one series of numbers -- it is no longer a DataFrame object. It is now a Series. A Python function ```type``` can be used to figure this information out:

In [ ]:
my_str = 'this is a string so its type should be "str"'

print(type(my_str))
print('\nWhat are the types of the other objects?\n\nFirst, the type of the entire ```close``` DataFrame is:\n')
print(type(close))
print('\nSecond, if I extract just one column (index) from this DataFrame, its type is:\n')
print(type(close['index']))

**So**, to be clear:

- ```DataFrames``` and ```Series``` are very similar except that Series has just one column. If you "extract" just one column from a DataFrame, Python will assume, by default, that you want to contain the resulting data in a Series object instead.
-  
The fact that a DataFrame automatically 'squeezes' down into a Series is sometimes inconvenient. If you want to insist that it doesn't do so, you can either use a ```list``` for columns (even if you have just one column in the list) or convert the Series back into a dataframe:

In [ ]:
print('This is still a DataFrame because we index with a list:\n')
display(close[['index']].tail())

print('This is a DataFrame because we convert the Series back into a DataFrame:\n')
display(close['index'].to_frame().tail())

## Viewing and slicing data

There are a few key methods for taking subsets of data:

- use the column name as a key (we already did this before)
  - If you have multiple levels, your key must be a ```tuple```
- .loc[] is used to get data based on *labels* (e.g., names of columns or dates in the index)
- .iloc[] is used to get data based on *indices*
- .xs() is used to get one cross-section of data based on a label

Slicing refers to taking slices of data between some start and end points.
- We can *slice* either by labels or indices

In [ ]:
close.loc['2024-01-08':'2024-01-10', ['AAPL', 'GOOGL']]

In [ ]:
close.iloc[10:13, :2]

Pandas is smart with dates and so you can get years or months etc. by just calling with partial date labels. Here, I don't specify the columns at all so I get all of them

In [ ]:
close.loc['2023-12']

## A warning about manipulating data

In the typical workflow, we often read some data and then make some changes to it
- Depending on what we are doing, we may want to go back to the original data

**A potential issue:** If you take a subset of data and make changes to it, you'll likely make changes to the *original* data because you haven't created a copy
- If you create a "new" DataFrame by writing something like ```new_data = old_data.iloc[:10]```, you are not creating a copy of the data
- Pandas is smart about memory and says "well, no need to duplicate all this information, ```new_data``` will just be a window (a "view") to this small part of ```old_data```
- This is great... but you **need** to know about it because otherwise you may end up with unexpected results
- Pandas might give you a warning when you make changes to "views" (e.g., if you try to modify ```new_data``` above) because Pandas knows that this might lead to unexpected behavior. We (probably) never really want to make changes to views.

---

If you see a warning about views, it means that (1) you are making changes through a view and (2) the operation might not be entirely safe

In [ ]:
# Create a copy of the dataframe so that I don't overwrite it here
close2 = close.copy()

data_2024 = close2.loc['2024']
print('\nHere is data_2024 right after we create this view:\n')
print(data_2024.head())
print('\nLet\'s change it a bit by setting a bunch of elements to zero and reprint data_2024:\n')
data_2024.iloc[:2,:2] = 0
print(data_2024.head())
print('\nLet\'s print the original dataframe, close2:\n')
print(close2.loc['2024'].head())

## Reshaping the data

If you want to rotate your data, you can TRANSPOSE it by calling the .T method -- we think of the data as being a matrix and this flips the rows and columns

In [ ]:
close.T

If you have a multiindex, you can use two commands for reshaping the data so that you move levels from the index to columns or vice versa:

1. Unstack moves levels from the index to columns. By default, you move the innermost level
2. Stack moves levels from the columns to the index. By default, you move the lowest level

Consider the original data:

In [ ]:
stock_data.head(5)

If I just call ```stack``` on the DataFrame, one of the levels in columns gets moved into the index
- Note: ```future_stack=True``` isn't important. It relates to the evolution of the Python language. If you remove this argument (and assuming that you're running a recent version of Pandas), you'll see why I added it.

In [ ]:
stock_data.stack(future_stack=True).head(5)

By default, ```stack()``` takes the innermost level from columns and moves it as the innermost level in index. 
- But we can change which level we want to move to the index:

In [ ]:
stock_data.stack(level=0, future_stack=True).head(5)

To illustrate unstacking, let me take what we have above and then unstack Dates to columns. 
- If you look at my code, you see that the beginning is exactly the same as above: I take the data and stack it.
- However, instead of terminating the line, I call a different method, ```unstack()```, on the result on the same lint
- This is called ```METHOD CHAINING```, which is something we have already done a bit. When we have multiple periods on a line, we are calling multiple methods 

In [ ]:
stock_data.stack(level=0, future_stack=True).unstack(level=0)

## Other ways of reshaping data and some notes

- You can use ```swaplevel``` method to swap levels
- You can use ```pivot``` method to create a new dataframe based on counts
- You can refer to different levels using the names of these levels instead of indices (always safer!)

## Let me compute returns as percentage changes in adj_close

There is a convenience method .pct_change() for doing so

In [ ]:
returns = close.pct_change()
returns.head()

If we want to view data based on some condition, we can filter the data 

For example, if we want to have days when AAPL's return is positive, we can create a column with True/False values by writing

```returns['AAPL'] > 0```

We can use this as an index and, in the style of Matlab, Pandas will understand that we are filtering columns

In [ ]:
returns[returns['AAPL'] > 0]

## Modifying DataFrames

We can modify DataFrames by, e.g.:

1. Creating new columns (we did this above)
2. Replacing values using .loc or .iloc (we did this above)
3. Replacing values based on some rule

We can also 'rename' columns or index

Starting from renaming columns, I might want to have company names instead of ticker symbols. I could use a dictionary to tell Python/Pandas what replacements I want to do:

In [ ]:
rename_dict = {'AAPL': 'Apple', 'GOOGL': 'Google', 'TSLA': 'Tesla'}
returns.rename(columns=rename_dict)

Notes:

1. Pandas is smart in that it notices that I'm giving it a dictionary
2. I did *not* set inplace=True, so we did not 'save' this modification to the dataframe

If we don't say "index" or "columns", we are making changes to the values in the DataFrame

Given the nature of these data, this is harder to demonstrate, but the example below shows that we can replace specific values with something else

In [ ]:
small_data = stock_data.iloc[:5,:5]
small_data

In [ ]:
value1 = small_data.iloc[1,2]
value2 = small_data.iloc[2,1]
rename_dict = {value1: -1000, value2: 1000}
small_data.replace(rename_dict)

## A few other common modifications

In some cases we want to take sums or ratios of DataFrames
- E.g., we might want to divide dataframe BE with dataframe ME

In some cases we might want to, say, divide a DataFrame with a Series
- We can either use +, -, *, and / symbols to do these operations or, if it is not 100% clear HOW we want to do the operations, we have .add, sub, .mul, and .div methods

For example, if I wanted to normalize the prices so that they all start at 100 and plot the data, I could do this:

In [ ]:
first_closes = close.iloc[0]
normalized_prices = 100 * close.div(first_closes, axis=1)
normalized_prices.plot(figsize=(12,8))

## GroupBy and Resample methods for computing statistics for groups of observations

Sometimes we want to group data based on some rule and compute, e.g., counts or means for these observations

For example, I could compute how many *observations* I have each month in the data.
- I first create a new column that contains just the month
- I then GroupBy by this column

Note: There is a lot going on on the first line where I create the yyyymm variable
- When I do ```returns.index```, I get labels from the index, e.g., 2020-01-02 and so forth
- Index has a method ```strftime()``` that allows me to create a string from something that represents time. I specify that I want to create a string of type yyyy-mm

In [ ]:
returns['yyyymm'] = returns.index.strftime('%Y-%m')
returns.head()

In [ ]:
returns.groupby('yyyymm')

In [ ]:
returns.groupby('yyyymm')[['AAPL','GOOGL']]

In [ ]:
grp = returns.groupby('yyyymm')[['AAPL','GOOGL']]
grp.last().head()

In [ ]:
grp.std().head()

If we have datetimes in the index, as we have here, we can RESAMPLE the data to different frequencies such as W=Week, MS=MonthStart, or ME=MonthEnd

If I resample, I can then take, e.g., means or sums of the data for each group or take the last value. To get the last price each month, we can write:

In [ ]:
returns.resample('ME').last().head()

## Creating, merging and joining data

We will work with existing data in this course

In terms of **creating** dataframes, it is useful to know that those can be created from all kinds of underlying data such as dictionaries

We often want to join different pieces of information together. When we merge or join data, we need to specify:

1. What are the variables or indexes by which we want to join and
2. What logic do we want to follow for keeping observations

We use commands merge, join, and concat for combining data

Because we will do this when we combine price information with accounting information, I won't give examples here

## Some statistics

Pandas has built-in methods for computing some basic statistics:

In [ ]:
# drop the yyyy-mm column that we created above to avoid some errors
returns = returns.drop(columns=['yyyymm'])

In [ ]:
print(returns.count(), end='\n\n')
print(returns.mean(), end='\n\n')
print(returns.std(), end='\n\n')
print(returns.skew(), end='\n\n')

If you wanted to compute the return on a trading rule that is always long Apple \\$1 and short Google \\$1, rebalancing daily, we could compute the realized **Sharpe ratio** of this strategy over our sample period with a few lines of code

In [ ]:
strategy_return = returns['AAPL'] - returns['GOOGL']
m = 252 * strategy_return.mean()
s = np.sqrt(252) * strategy_return.std()
sr = m / s
print(f'The annualized Sharpe ratio to three decimal points is {sr:.3f}')

## A practical example of the power of Pandas and method chaining

The benefit of Pandas is that you can get *a lot* done with just one line of code through method chaining

For example, if I start with the original data and I want to plot rolling 21-day volatilities for just AAPL and GOOGL, I could do the following
- Note: ```.rolling(window=21)``` takes each point in time and uses 21 observations (~one month in trading days) to create a group. When we call ```.std()``` on this group, we get the volatility (= standard deviation) estimated using one month of data to each date 

In [ ]:
data = stock_data.xs('Close', axis=1, level=0)
data = data[['AAPL','GOOGL']]
ret = data.pct_change()
vols = ret.rolling(window=21).std()
vols.plot()

But I know the basic ingredients and am fluent enough in it, it might make sense to do this on just one line

We read this from left to right so this isn\'t that bad. When we have a dot (.), we know that something is being applied to whatever was created on the left

In [ ]:
stock_data.xs('Close', axis=1, level=0)[['AAPL','GOOGL']].pct_change().rolling(window=21).std().plot(figsize=(12,8))

This is pretty readable. However, please do **not** use method chaining on one line when it hurts readability 

If you want to do method chaining -- which is great if you don't want to save the intermediary steps -- you can often format your code differently to get it done AND enhance readability. For example, we could write the same code like this: 

In [ ]:
(
    stock_data
    .xs('Close', axis=1, level=0)  # Select 'Close' from the multi-index columns
    [['AAPL', 'GOOGL']]            # Filter for the desired stocks
    .pct_change()                  # Calculate percentage change
    .rolling(window=21)            # Apply rolling window of 21
    .std()                         # Compute rolling standard deviation
    .plot(figsize=(12, 8))         # Plot with specified figure size
)

Here, we wrap the entire expression inside parentheses, which tells Python that we, in effect, have just one line here no matter what the formatting is

## Saving and loading data

Pandas supports multiple data formats, such as csv and Excel, with built-in methods. With these methods you can specify how the input data need to be interpreted.

There are other formats such as pickle, json, and parquet for storing objects.
- Every file format has different pros and cons

Pickle is very popular because it is a really simple way of storing dictionaries and DataFrames
- A popular use case would be organize all your data into a big dictionary of, e.g., DataFrames and other objects and then save that as a pickle file
- When you read it, you'll have exactly the same dictionary back
- If you were to use, e.g., a CSV file, you would need redefine the indexes and every piece of data would likely be in its own file

The downside of pickle is that it has backwards-compatibility issues:
- A pickle file created in one version of Python/Pandas might not be (= typically is not) readable in another version 
- So be careful if you want to share files
- In an enterprise applications the development environment is typically managed so that *everything* in your Python installation stays the same
  - In this case, you don't have to worry about incompatibility
  - The only issue is that when you want to transition to a newer environment, you may need to port over any pickle files on which you rely
  
If I wanted to write and read dataframes directly from Pandas, it would be really simple:

In [ ]:
returns.to_pickle('./returns.pkl')
returns.head()

In [ ]:
returns2 = pd.read_pickle('./returns.pkl')
returns2.head()